In [18]:
import json
import numpy as np
from bottom_up.all import pick_bfs_connected
from bottom_up.all import get_lr
from bottom_up.all import get_dependents
from bottom_up.all import get_f1
from bottom_up.all import featurize_ultra_local
from bottom_up.all import get_local_feats
from bottom_up.all import get_global_feats
from bottom_up.all import featurize_parent_proposal
from bottom_up.all import pick_l2r_connected
from bottom_up.all import featurize_child_proposal
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from code.printers import pretty_print_conl
from bottom_up.all import pick_bfs_connected
from bottom_up.all import len_tree
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import pickle
with open("a", "rb") as inf:
    dt = pickle.load(inf)
    
clf, vectorizer = dt

/Users/ahandler/.virtualenvs/p3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.20.2 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/ahandler/.virtualenvs/p3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator DictVectorizer from version 0.20.2 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [410]:
def pick_at_random(l, **kwargs):
    return random.sample(l, 1)[0]

from code.treeops import bfs
from bottom_up.all import pick_bfs
from bottom_up.all import get_dependents_and_governors
import random

def runtime_path_wild_frontier(sentence, pi, clf, vectorizer, verbose=False):
    T = {i for i in sentence["q"]}
    F = set()
    
    # init frontier
    for v in T:
        for i in sentence["tokens"]:
            F.add(i["index"])

    d, heads, c = bfs(g=sentence, hop_s=0) 
    while len(F) > 0:
        vertex = pi(F=F, d=d, T=T, s=sentence)

        if vertex != 0: # bug here?
            feats = get_local_feats(vertex=vertex, sentence=sentence, d=d, current_tree=T)
            feats = get_global_feats(sentence=sentence, feats=feats, vertex=vertex, current_tree=T)

            X = vectorizer.transform([feats])
            pred = clf.predict_proba(X)[0]
            y = clf.predict(X)[0]

            if y == 1:
                T.add(vertex)
                if vertex != 0:
                    for i in get_dependents_and_governors(vertex, sentence, T):
                        if i not in T and i is not None:
                            F.add(i)
                else:
                    for i in get_dependents(sentence, vertex):
                        if i["dependent"] not in T and i is not None:
                            F.add(i)
        F.remove(vertex)

    return T

tot = 0
for pno, paths in enumerate(validation_paths):
    paths=json.loads(paths)
    sentence = paths["sentence"]
    predicted = runtime_path_wild_frontier(sentence, pi=pick_l2r_connected, clf=clf, vectorizer=vectorizer)
    f1s = get_f1(predicted, sentence)
    tot += f1s
print(tot/len(validation_paths))

0.8338651182073651


In [487]:
from bottom_up.all import get_labels_and_features
from bottom_up.all import get_governor

import json
import pickle
import numpy as np
from bottom_up.all import get_lr
from bottom_up.all import get_dependents
from bottom_up.all import get_f1
from bottom_up.all import featurize_ultra_local
from bottom_up.all import featurize_parent_proposal
from bottom_up.all import featurize_child_proposal
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from code.printers import pretty_print_conl
from bottom_up.all import len_tree

training_paths = [_ for _ in open("training.paths")]
validation_paths = [_ for _ in open("validation.paths")]


vectorizer = DictVectorizer(sparse=True)

train_features, train_labels = get_labels_and_features(training_paths)

X_train = vectorizer.fit_transform(train_features)

y_train = np.asarray(train_labels)

val_features, val_labels = get_labels_and_features(validation_paths)

X_val = vectorizer.transform(val_features)

y_val = np.asarray(val_labels)

clf = LogisticRegression(random_state=0,
                         solver='lbfgs',
                         C=.5,
                         multi_class='ovr').fit(X_train, y_train)

print(clf.score(X_val,y_val))
print(clf.score(X_train, y_train))

din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din
din


KeyboardInterrupt: 

In [488]:
confusion = []
from collections import Counter
from collections import defaultdict
all_vals = defaultdict(list)
scr = clf.predict(X_train)
o = 0
h = 0
tot = []
big_list = []
big_list_sentences = []

for paths in training_paths:
    paths=json.loads(paths)
    s = paths["sentence"]
    for p in paths["paths"]:
        current_tree, vertex, decision = p
        if vertex != 0:  
            big_list.append(p)
            big_list_sentences.append(s)
        

for ino, i in enumerate(train_features):
    if i["disconnected"]:
        o += 1
        if scr[ino] == y_train[ino]:
            h += 1
        else:
            tot.append((scr[ino], y_train[ino], big_list_sentences[ino], big_list[ino]))
print(h/(len(tot) + h))
print(len(tot))

0.9894657919400187
1405


In [505]:
hits = 0
govs = defaultdict(int)
for t in tot[500:]:

    pred, was, s, path = t
    current_tree, vertex, decision = path
    current_tree.sort()
    governed = [_["dep"] for _ in s["basicDependencies"] if _["dependent"] == vertex]
    for g in governed:
        govs[g] += 1
    if g == "case":
        break
        
print(govs)
print(hits)
print(len(tot))
print(sum(govs.values()))

defaultdict(<class 'int'>, {'det': 1, 'amod': 1, 'compound': 2, 'case': 1})
0
1405
5


In [506]:
print(current_tree, vertex, pred, was)
pretty_print_conl(s)
print(s["compression_indexes"])

[12, 13, 14, 15, 16, 17] 20 0 1
1	Chairman	<-nsubj-	asserts
2	of	<-case-	Aksa
3	energy	<-compound-	Aksa
4	company	<-compound-	Aksa
5	Aksa	<-nmod-	Chairman
6	,	<-punct-	Chairman
7	Cemil	<-compound-	Kazanci
8	Kazanci	<-appos-	Chairman
9	,	<-punct-	Chairman
10	asserts	<-ROOT-	ROOT
11	that	<-mark-	interested
12	Aksa	<-nsubj-	interested
13	may	<-aux-	interested
14	be	<-cop-	interested
15	interested	<-ccomp-	asserts
16	in	<-mark-	placing
17	placing	<-advcl-	interested
18	a	<-det-	bid
19	bid	<-dobj-	placing
20	in	<-case-	tender
21	the	<-det-	tender
22	new	<-amod-	tender
23	nuclear	<-amod-	tender
24	power	<-compound-	tender
25	plant	<-compound-	tender
26	tender	<-nmod-	bid
27	.	<-punct-	asserts
[12, 13, 14, 15, 16, 17, 20, 21, 22, 23, 24, 25, 26]
